# 05 - Topic Modeling - Exercise 3

In [1]:
import pandas as pd
import numpy as np
import gensim

C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
### ONLY RUN ONCE ####
# import nltk
# nltk.download('wordnet')

To get good results when performing topic modeling, the documents should be large enough. To avoid emails that are too short, we **group the emails by sender** (using the SenderPersonId feature) and then consider each document to be the ExtractedSubject + ExtractedBodyText of all the emails belonging to each sender. We also discard emails where both the ExtratedSubject and the ExtractedBodyText are NaN.

Another option would be to group the emails by subject. However, the ExtratedSubject column has a considerable larger number of NaNs than the column SenderPersonId. Thus, we chose to group the emails by sender.

We then apply our text **pre-processing pipeline** to each document: tokenization, stopwords and digits/numbers removal, lemmatization and removal of tokens that are too small.

In [3]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# create DataFrame
path_emails = 'hillary-clinton-emails/Emails.csv'
emails = pd.read_csv(path_emails, usecols = ['SenderPersonId', 'ExtractedSubject', 'ExtractedBodyText'])

# drop records where both ExtractedSubject and ExtractedBodyText are NaN
emails.dropna(axis=0, thresh=1, subset=[['ExtractedSubject', 'ExtractedBodyText']], inplace=True)
emails.fillna(' ', inplace = True)

# group emails by sender
emails_raw = []

for sender in emails.groupby('SenderPersonId'):
    grouped_emails = ''
    
    for subject, body in zip(sender[1]['ExtractedSubject'], sender[1]['ExtractedBodyText']):
        grouped_emails += (subject + ' ' + body + ' ')
    
    emails_raw.append(grouped_emails)

# tokenization
email_tokens = []

for email in emails_raw:
    email_tokens.append(regexp_tokenize(email, pattern='\w+'))

# remove stopwords and digits/numbers
stop_words = set(stopwords.words('english'))

# also consider as stopwords those words typically related to emails
stopwords_emails = ['fyi', 'fm', 'am', 'pm', 'n\'t', 'sent', 'from', 'to', 'subject', 'fw', 'fwd', 'fvv',
                    'cc', 'bcc', 'attachments', 're', 'date', 'html', 'php']

stop_words.update(stopwords_emails)

email_clean_tokens = []

for email in email_tokens:
    clean_tokens = [token for token in email if (token.lower() not in stop_words 
                                                 and any(char.isdigit() for char in token) == False)]
    email_clean_tokens.append(clean_tokens)

# lemmatization
wnl = WordNetLemmatizer()

email_lemma = []

# normalization also takes place in this step
for email in email_clean_tokens:
    lemma = [wnl.lemmatize(token.lower()) for token in email]
    email_lemma.append(lemma)
    
# removal of tokens that are too small
email_clean = []

for email in email_lemma:
    clean = [token for token in email if len(token) > 1]
    email_clean.append(clean)

We initialize a dictionary using the pre-processed corpus. The dictionary encapsulates the mapping between the words in those documents and their integer IDs. We then convert the collection of words in each document to its bag-of-words representation.

In [4]:
# create dictionary (mapping between words and IDs)
dictionary = gensim.corpora.Dictionary(email_clean)

# bag-of-words representation of each document in the corpus
corpus = [dictionary.doc2bow(email) for email in email_clean]

We perform topic modeling over the corpus by running a LDA model. We run the model for different numbers of topics and we display the 10 most significant words of each topic.

In [5]:
number_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for n in number_topics:
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word=dictionary, passes=10)
   
    print('#### Model with', n, 'topics ####')
   
    topic_id = 1
   
    for topic in lda_model.show_topics(num_topics=n, num_words=10, log=False, formatted=False):
        
        string_words = ''
        
        for word in topic[1]:
            string_words += (word[0] + ' ')
        
        print('topic #', topic_id, ':', string_words)

        topic_id += 1

#### Model with 5 topics ####
topic # 1 : state would woman also department one agreement government work new 
topic # 2 : call state gov talk want tomorrow pls get know also 
topic # 3 : obama would american president party one said republican new state 
topic # 4 : secretary office state meeting room department en arrive route depart 
topic # 5 : state clinton said one would haiti cheryl mill people work 
#### Model with 10 topics ####
topic # 1 : state call gov cheryl would mill haiti talk secretary said 
topic # 2 : state clinton one said new would policy time president obama 
topic # 3 : state would new senate vote secretary treaty call start department 
topic # 4 : woman state would afghan afghanistan said issue one new could 
topic # 5 : state department official local gov information mail sensitive case intergovernmental 
topic # 6 : state call gov see com work would also know get 
topic # 7 : un settlement would china people work haitian could one chinese 
topic # 8 : obama he

Picking a small number of topics might lead to topics which most significant words don't have much in common. Picking a large number of topics might lead to redundant topics (i.e. topics with the similar significant words). There's no precise way to pick the number of topics, but in this case **25 topics** is a good compromise.